In [2]:
%pip install ollama
%pip install neo4j

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ollama
# This checks if the notebook can see the models in your app
print(ollama.list())

models=[Model(model='gemma3:4b', modified_at=datetime.datetime(2026, 1, 28, 9, 31, 18, 52648, tzinfo=TzInfo(19800)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='4.3B', quantization_level='Q4_K_M'))]


In [ ]:
# Working version for Alex Developer
# import ollama
# from neo4j import GraphDatabase

# # --- CONFIGURATION ---
# URI = "neo4j://127.0.0.1:7687"
# USER = "neo4j"
# PASSWORD = "12345678" 

# def generate_ai_assessment(occupation_name):
#     # Initialize connection
#     driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
#     with driver.session() as session:
#         # FIXED QUERY: 
#         # 1. Matches the Job (Software Analyst)
#         # 2. Finds its required skills
#         # 3. Finds candidates linked to those SAME skills
#         query = """
#         MATCH (o:Occupation) 
#         WHERE o.name =~ $job_name
        
#         // Find skills linked to this job
#         MATCH (o)-[:REQUIRES]->(s:Skill)
        
#         // Find candidates linked to those specific skills
#         // We use <-[:HAS_SKILL]- because candidates point TO skills
#         MATCH (s)<-[:HAS_SKILL]-(c:Candidate)
        
#         WITH o, c, collect(DISTINCT s.name) AS matched_skills
#         RETURN c.name AS candidate, 
#                o.name AS job,
#                matched_skills,
#                [(o)-[:REQUIRES]->(req) | req.name] AS all_required
#         ORDER BY size(matched_skills) DESC
#         LIMIT 1
#         """
        
#         # Use .data() to avoid the "Record object" formatting issues
#         results = session.run(query, job_name=f"(?i).*{occupation_name}.*").data()
        
#         if not results:
#             return f"❌ No candidates found in the graph who match the skills for '{occupation_name}'."

#         result = results[0]
#         candidate_name = result['candidate']
#         job_title = result['job']
#         matched = result['matched_skills']
#         all_reqs = result['all_required']

#         # 2. Prepare the data for Gemma 3
#         # Calculate what the candidate is missing
#         missing_skills = list(set(all_reqs) - set(matched))
        
#         prompt = f"""
#         You are a Technical Recruiter. 
#         Candidate: {candidate_name}
#         Target Job: {job_title}
#         Matched Skills: {matched}
#         Missing Skills from Job Requirements: {missing_skills[:5]}

#         Explain why this candidate is a good match despite missing some skills. 
#         Focus on the 'Semantic overlap' (e.g., if they have SQL, they can learn DBMS).
#         """

#         # 3. Call Gemma 3 4B (The 'Thinking' Model)
#         print(f"🤖 Gemma 3 is analyzing {candidate_name}...")
#         response = ollama.generate(model='gemma3:4b', prompt=prompt)
        
#         return response['response']

# # Run the fixed script
# print(generate_ai_assessment("software analyst"))

In [ ]:
import ollama
from neo4j import GraphDatabase

# --- CONFIGURATION ---
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "12345678" # Ensure this is correct

def generate_ai_assessment(occupation_name):
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    with driver.session() as session:
        query = """
        MATCH (o:Occupation) 
        WHERE o.name =~ $job_name

        // 1. Find the required skills
        MATCH (o)-[:REQUIRES]->(s:Skill)

        // 2. Find connected candidates
        MATCH (s)<-[:HAS_SKILL]-(c:Candidate)
        
        // 3. Aggregate results per candidate
        WITH o, c, collect(DISTINCT s.name) AS matched_skills_list
        
        // 4. Get all required skills for the job to show missing ones
        MATCH (o)-[:REQUIRES]->(all_s:Skill)
        WITH o, c, matched_skills_list, collect(DISTINCT all_s.name) AS all_required_list

        // 5. The Fix: We use head() in case ID is a list, and elementId as a fallback
        RETURN o.name AS job, 
               coalesce(toString(c.id), toString(elementId(c))) AS candidate, 
               matched_skills_list AS matched_skills,
               all_required_list AS all_required
        ORDER BY size(matched_skills_list) DESC
        LIMIT 1
        """
        
        results = session.run(query, job_name=f"(?i).*{occupation_name}.*").data()
        
        # If no candidates matched ANY skills, the query above returns empty.
        # We handle that fallback here in Python.
        if not results:
            return "No candidates in the database have any skills matching this role's requirements."

        res = results[0]
        candidate_id = res['candidate']
        job_title = res['job']
        matched = res['matched_skills']
        all_reqs = res['all_required']
        missing = list(set(all_reqs) - set(matched))

        print(f"🤖 Gemma 3 is analyzing Candidate ID: {candidate_id} for {job_title}...")

        prompt = f"""
        Job: {job_title}
        Candidate ID: {candidate_id}
        Matched Skills: {matched}
        Missing Skills: {missing[:10]}
        
        Task: Summarize why this candidate is a fit and what they should learn next.
        """
        
        response = ollama.generate(model='gemma3:4b', prompt=prompt)
        return response['response']

# Try it now!
print(generate_ai_assessment("software tester"))

In [9]:
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
print(driver.execute_query("MATCH (o:Occupation) WHERE o.name =~ '(?i).*software.*' RETURN o.name LIMIT 1").records)

[<Record o.name='software analyst'>]


In [ ]:
# import ollama
# from neo4j import GraphDatabase

# # --- CONFIGURATION ---
# URI = "neo4j://127.0.0.1:7687"
# USER = "neo4j"
# PASSWORD = "12345678" 
# driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# def sync_embeddings():
#     with driver.session() as session:
#         # 1. We fetch the internal ID (elementId) so we don't have to guess property names
#         nodes = session.run("""
#             MATCH (n) WHERE n:Candidate OR n:Skill 
#             RETURN elementId(n) as internal_id, 
#                    n.name as name, 
#                    labels(n)[0] as label, 
#                    n.category as cat,
#                    n.id as id
#         """).data()
        
#         for node in nodes:
#             # Determine text: Candidates use 'cat', Skills use 'name'
#             text = node['name'] if node['label'] == 'Skill' else node['cat']
            
#             # Identify the display name for the print statement
#             display_name = node['name'] if node['name'] else f"ID:{node['id']}"
            
#             if not text:
#                 continue
            
#             # Generate embedding
#             res = ollama.embed(model='nomic-embed-text', input=text)
#             embedding = res['embeddings'][0]
            
#             # 2. Update using elementId (the most reliable way)
#             session.run("""
#                 MATCH (n) WHERE elementId(n) = $internal_id
#                 SET n.graph_embedding = $emb, n.embedding = $emb
#             """, internal_id=node['internal_id'], emb=embedding)
            
#             print(f"✅ Synced {node['label']} [{display_name}] (Size: {len(embedding)})")

# sync_embeddings()

In [60]:
#Working version for single output 
import ollama
from neo4j import GraphDatabase

# --- CONFIGURATION ---
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "12345678" 

def generate_ai_assessment(occupation_name):
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    with driver.session() as session:
        query = """
        MATCH (o:Occupation) 
        WHERE o.name =~ $job_name

        // 1. Find the required skills for the job
        MATCH (o)-[:REQUIRES]->(s:Skill)

        // 2. Find connected candidates via the similarity links we built
        MATCH (s)<-[:HAS_SKILL]-(c:Candidate)
        
        // 3. Aggregate matches per candidate
        WITH o, c, collect(DISTINCT s.name) AS matched_skills_list
        
        // 4. Get the full list of job requirements to calculate what's missing
        MATCH (o)-[:REQUIRES]->(all_s:Skill)
        WITH o, c, matched_skills_list, collect(DISTINCT all_s.name) AS all_required_list

        // 5. THE FIX: Coalesce handles the ID property vs Internal Element ID
        RETURN o.name AS job, 
               coalesce(toString(c.id), elementId(c)) AS candidate, 
               matched_skills_list AS matched_skills,
               all_required_list AS all_required
        ORDER BY size(matched_skills_list) DESC
        LIMIT 1
        """
        
        results = session.run(query, job_name=f"(?i).*{occupation_name}.*").data()
        
        if not results:
            return "No candidates found with matching skills for this role."

        res = results[0]
        candidate_id = res['candidate']
        display_id = candidate_id.split(':')[0] if ":" in candidate_id else candidate_id
        job_title = res['job']
        matched = res['matched_skills']
        all_reqs = res['all_required']
        
        # Calculate missing skills using set subtraction
        missing = list(set(all_reqs) - set(matched))

        print(f"🤖 Gemma 3 is analyzing Candidate ID: {display_id} for {job_title}...")

        # Updated prompt for better reasoning from Gemma 3
        prompt = f"""
        Role: Expert Technical Recruiter
        Job Title: {job_title}
        Candidate ID: {candidate_id}
        
        Skills the candidate has: {', '.join(matched)}
        Skills the candidate lacks: {', '.join(missing[:10])}
        
        Task:
        1. Summarize the candidate's strength for this specific role.
        2. Identify the most critical gap they need to fill.
        3. Provide a 1-sentence 'verdict' on their readiness.
        """
        
        response = ollama.generate(model='gemma3:4b', prompt=prompt)
        return response['response']

# Run the assessment
print(generate_ai_assessment("cloud software developer"))

SyntaxError: invalid syntax (2532914028.py, line 9)

In [61]:
import ollama
from neo4j import GraphDatabase

# --- CONFIGURATION ---
URI = "neo4j://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "12345678" 

def generate_top_candidates_report(occupation_name):
    driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))
    
    with driver.session() as session:
        query = """
        MATCH (o:Occupation) 
        WHERE o.name =~ $job_name
        MATCH (o)-[:REQUIRES]->(s:Skill)
        MATCH (s)<-[:HAS_SKILL]-(c:Candidate)
        
        // 1. Group by candidate and count matched skills
        WITH o, c, collect(DISTINCT s.name) AS matched_skills_list
        
        // 2. Get full requirements for the job
        MATCH (o)-[:REQUIRES]->(all_s:Skill)
        WITH o, c, matched_skills_list, collect(DISTINCT all_s.name) AS all_required_list
        
        // 3. Prepare result with clean IDs and skill counts
        RETURN o.name AS job, 
               coalesce(toString(c.id), last(split(elementId(c), ':'))) AS candidate_id, 
               size(matched_skills_list) AS match_count,
               matched_skills_list AS matched_skills,
               all_required_list AS all_required
        ORDER BY match_count DESC
        LIMIT 5
        """
        
        results = session.run(query, job_name=f"(?i).*{occupation_name}.*").data()
        
        if not results:
            return "No matching candidates found for this comparison."

        # Process the list for the AI
        job_title = results[0]['job']
        candidate_summaries = []

        for res in results:
            missing = list(set(res['all_required']) - set(res['matched_skills']))
            summary = (
                f"Candidate {res['candidate_id']}:\n"
                f"- Matched Skills ({res['match_count']}): {', '.join(res['matched_skills'])}\n"
                f"- Key Missing Skills: {', '.join(missing[:3])}"
            )
            candidate_summaries.append(summary)

        print(f"🤖 Gemma 3 is comparing {len(results)} candidates for {job_title}...")

        # Construct a comparative prompt
        prompt = f"""
        Role: Senior Talent Acquisition Lead
        Job: {job_title}
        
        Candidates Shortlist:
        {chr(10).join(candidate_summaries)}
        
        Task:
        1. Rank these candidates from 1 to 5.
        2. Briefly explain why the #1 candidate stands out.
        3. Identify a 'dark horse' candidate (someone with unique potential despite missing skills).
        4. Provide a 1-sentence recommendation for the hiring manager.
        """
        
        response = ollama.generate(model='gemma3:4b', prompt=prompt)
        return response['response']

# Run the comparative study
print(generate_top_candidates_report("software tester"))

🤖 Gemma 3 is comparing 5 candidates for software tester...
Okay, let’s analyze these candidates for the Software Tester role, considering the provided information.

**1. Ranking the Candidates (1-5)**

1.  **Candidate 17943**
2.  **Candidate 17**
3.  **Candidate 0**
4.  **Candidate 7**
5.  **Candidate 10**


**2. Why Candidate 17943 Stands Out (#1)**

Candidate 17943 has the highest number of matched skills (52) across a *broad* range of technologies. While there are gaps, the sheer volume of programming languages, tools, and methodologies listed suggests a generally well-rounded technical background. This breadth of experience could translate well to understanding diverse software systems and testing approaches. The listed skills also align somewhat with modern software development practices, which is a plus.

**3. Identifying a ‘Dark Horse’ Candidate**

**Candidate 10** is the dark horse candidate.  While the skill set is largely the same as the others, the *diversity* of languages a